In [5]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn; seaborn.set_style('whitegrid')
import numpy as np

from pomegranate import *

np.random.seed(0)
np.set_printoptions(suppress=True)

In [4]:
#!pip install -U numpy

## CG rich region identification example

Lets take the simplified example of CG island detection on a sequence of DNA. DNA is made up of the four canonical nucleotides, abbreviated 'A', 'C', 'G', and 'T'. We can say that regions of the genome that are enriched for nucleotides 'C' and 'G' are 'CG islands', which is a simplification of the real biological concept but sufficient for our example. The issue with identifying these regions is that they are not exclusively made up of the nucleotides 'C' and 'G', but have some 'A's and 'T's scatted amongst them. A simple model that looked for long stretches of C's and G's would not perform well, because it would miss most of the real regions.

We can start off by building the model. Because HMMs involve the transition matrix, which is often represented using a graph over the hidden states, building them requires a few more steps that a simple distribution or the mixture model. Our simple model will be composed of two distributions. One distribution wil be a uniform distribution across all four characters and one will have a preference for the nucleotides C and G, while still allowing the nucleotides A and T to be present.

In [14]:
d1 = DiscreteDistribution({'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25})
d2 = DiscreteDistribution({'A': 0.10, 'C': 0.40, 'G': 0.40, 'T': 0.10}) # << C i G występują tutaj częściej

In [15]:
# Definiujemy dwa stany
s1 = State(d1, name='background')
s2 = State(d2, name='CG island')

In [16]:
# Tworzymy model HMM
model = HiddenMarkovModel()
model.add_states(s1, s2)

In [17]:
# Dodajemy prawdopodbieństwa przejścia ze stanu do stanu
model.add_transition(model.start, s1, 0.5)
model.add_transition(model.start, s2, 0.5)
model.add_transition(s1, s1, 0.9)
model.add_transition(s1, s2, 0.1)
model.add_transition(s2, s1, 0.1)
model.add_transition(s2, s2, 0.9)

Now, finally, we need to bake the model in order to finalize the internal structure. Bake must be called when the model has been fully specified.

In [18]:
model.bake()

In [19]:
# Uwaga! Indeksy nie odziweciedlają kolejności dodawania stanów
seq = numpy.array(list('CGACTACTGACTACTCGCCGACGCGACTGCCGTCTATACTGCGCATACGGC'))

hmm_predictions = model.predict(seq)

print("sequence: {}".format(''.join(seq)))
print("hmm pred: {}".format(''.join(map( str, hmm_predictions))))

sequence: CGACTACTGACTACTCGCCGACGCGACTGCCGTCTATACTGCGCATACGGC
hmm pred: 111111111111111000000000000000011111111111111110000


> The predicted integers don't correspond to the order in which states were added to the model, but rather, the order that they exist in the model after a topological sort. 

Let's say, though, that we want to get rid of that CG island prediction at the end because we don't believe that real islands can occur at the end of the sequence. We can take care of this by adding in an explicit end state that only the non-island hidden state can get to. We enforce that the model has to end in the end state, and if only the non-island state gets there, the sequence of hidden states must end in the non-island state. Here's how:

In [22]:
model = HiddenMarkovModel()
model.add_states(s1, s2)
model.add_transition(model.start, s1, 0.5)
model.add_transition(model.start, s2, 0.5)
model.add_transition(s1, s1, 0.89 )
model.add_transition(s1, s2, 0.10 )
model.add_transition(s1, model.end, 0.01)
model.add_transition(s2, s1, 0.1)
model.add_transition(s2, s2, 0.9)
model.bake()

Note that all we did was add a transition from s1 to model.end with some low probability. This probability doesn't have to be high if there's only a single transition there, because there's no other possible way of getting to the end state.

In [23]:
seq = numpy.array(list('CGACTACTGACTACTCGCCGACGCGACTGCCGTCTATACTGCGCATACGGC'))

hmm_predictions = model.predict(seq)

print("sequence: {}".format(''.join(seq)))
print("hmm pred: {}".format(''.join(map( str, hmm_predictions))))


sequence: CGACTACTGACTACTCGCCGACGCGACTGCCGTCTATACTGCGCATACGGC
hmm pred: 111111111111111000000000000000011111111111111111111


In the same way that mixtures could provide probabilistic estimates of class assignments rather than only hard labels, hidden Markov models can do the same. These estimates are the posterior probabilities of belonging to each of the hidden states given the observation, but also given the rest of the sequence.

In [24]:
print(model.predict_proba(seq)[12:19])

[[0.19841088 0.80158912]
 [0.32919701 0.67080299]
 [0.38366073 0.61633927]
 [0.58044619 0.41955381]
 [0.69075524 0.30924476]
 [0.74653183 0.25346817]
 [0.76392808 0.23607192]]
